In [1]:
import os
import sys
import time
import glob
import datetime
import sqlite3
import numpy as np

In [34]:
msd_subset_path='./MillionSongSubset/'
msd_subset_data_path=os.path.join(msd_subset_path,'data')
msd_subset_addf_path=os.path.join(msd_subset_path,'AdditionalFiles')
assert os.path.isdir(msd_subset_path),'wrong path'

In [35]:
msd_code_path='./MSongsDB/'
assert os.path.isdir(msd_code_path),'wrong path'

In [36]:
sys.path.append( os.path.join(msd_code_path,'PythonSrc') )

In [37]:
import hdf5_getters as GETTERS

In [38]:
def strtimedelta(starttime,stoptime):
    return str(datetime.timedelta(seconds=stoptime-starttime))

In [39]:
def apply_to_all_files(basedir,func=lambda x: x,ext='.h5'):
    """
    From a base directory, go through all subdirectories,
    find all files with the given extension, apply the
    given function 'func' to all of them.
    If no 'func' is passed, we do nothing except counting.
    INPUT
       basedir  - base directory of the dataset
       func     - function to apply to all filenames
       ext      - extension, .h5 by default
    RETURN
       number of files
    """
    cnt = 0
    # iterate over all files in all subdirectories
    for root, dirs, files in os.walk(basedir):
        files = glob.glob(os.path.join(root,'*'+ext))
        # count files
        cnt += len(files)
        # apply function to all files
        for f in files :
            func(f)       
    return cnt

In [40]:
# we can now easily count the number of files in the dataset
print 'number of song files:',apply_to_all_files(msd_subset_data_path)

number of song files: 10000


In [47]:
all_artist_names = []

In [48]:
# we define the function to apply to all files
def func_to_get_artist_name(filename):
    """
    This function does 3 simple things:
    - open the song file
    - get artist ID and put it
    - close the file
    """
    h5 = GETTERS.open_h5_file_read(filename)
    artist_name = GETTERS.get_artist_name(h5)
    all_artist_names.append( artist_name )
    h5.close()

In [49]:
t1 = time.time()
apply_to_all_files(msd_subset_data_path,func=func_to_get_artist_name)
t2 = time.time()
print 'all artist names extracted in:',strtimedelta(t1,t2)

all artist names extracted in: 0:01:00.827548


In [50]:
print 'found',len(all_artist_names),'unique artist names'
for k in range(5):
    print list(all_artist_names)[k]

found 10000 unique artist names
My Little Airport
Porno For Pyros
Oscar Peterson Trio
Robben Ford
The Ronettes


In [46]:
dir(GETTERS)


['__builtins__',
 '__doc__',
 '__file__',
 '__name__',
 '__package__',
 '__warningregistry__',
 'get_analysis_sample_rate',
 'get_artist_7digitalid',
 'get_artist_familiarity',
 'get_artist_hotttnesss',
 'get_artist_id',
 'get_artist_latitude',
 'get_artist_location',
 'get_artist_longitude',
 'get_artist_mbid',
 'get_artist_mbtags',
 'get_artist_mbtags_count',
 'get_artist_name',
 'get_artist_playmeid',
 'get_artist_terms',
 'get_artist_terms_freq',
 'get_artist_terms_weight',
 'get_audio_md5',
 'get_bars_confidence',
 'get_bars_start',
 'get_beats_confidence',
 'get_beats_start',
 'get_danceability',
 'get_duration',
 'get_end_of_fade_in',
 'get_energy',
 'get_key',
 'get_key_confidence',
 'get_loudness',
 'get_mode',
 'get_mode_confidence',
 'get_num_songs',
 'get_release',
 'get_release_7digitalid',
 'get_sections_confidence',
 'get_sections_start',
 'get_segments_confidence',
 'get_segments_loudness_max',
 'get_segments_loudness_max_time',
 'get_segments_loudness_start',
 'get_seg